# Build Simple Tools
- **SQL Functions**: Create queries that access data critical to steps in the customer service workflow for processing a return.
- **Simple Python Function**: Create and register a Python function to overcome some common limitations of language models.

In [0]:
%pip install -qqqq -U -r requirements.txt
# Restart to load the packages into the Python environment
dbutils.library.restartPython()

In [0]:
%run ../00_setup/config

In [0]:
spark.sql(f"USE CATALOG `{catalog_name}`")
spark.sql(f"USE SCHEMA  `{schema_name}`")

In [0]:
%sql
SELECT current_catalog(), current_schema()

# Customer Service Return Processing Workflow

Below is a structured outline of the **key steps** a customer service agent would typically follow when **processing a return**. This workflow ensures consistency and clarity across your support team.

---

## 1. Get the Latest Return in the Processing Queue
- **Action**: Identify and retrieve the most recent return request from the ticketing or returns system.  
- **Why**: Ensures you’re working on the most urgent or next-in-line customer issue.

---

In [0]:
%sql
SELECT * FROM databricks_workshop.jywu.customer_services 
LIMIT 5

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_latest_return()
RETURNS TABLE(purchase_date DATE, issue_category STRING, issue_description STRING, name STRING)
COMMENT 'Returns the most recent customer service interaction, such as returns.'
RETURN (
  SELECT 
    CAST(date_time AS DATE) AS purchase_date,
    issue_category,
    issue_description,
    name
  FROM databricks_workshop.jywu.customer_services
  ORDER BY date_time DESC
  LIMIT 1
);

In [0]:
%sql
SELECT * FROM get_latest_return()

---

## 2. Use the User Name to Look Up the Order History
- **Action**: Query your order management system or customer database using the Username.  
- **Why**: Reviewing past purchases, return patterns, and any specific notes helps you determine appropriate next steps (e.g., confirm eligibility for return).

###### Note: We've doing a little trick to give the LLM extra context into the current date by adding todays_date in the function's response
---

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_order_history(user_name STRING)
RETURNS TABLE (returns_last_12_months INT, issue_category STRING, todays_date DATE)
COMMENT 'This takes the user_name of a customer as an input and returns the number of returns and the issue category'
LANGUAGE SQL
RETURN 
SELECT count(*) as returns_last_12_months, issue_category, now() as todays_date
FROM databricks_workshop.jywu.customer_services 
WHERE name = user_name 
GROUP BY issue_category;

In [0]:
%sql
SELECT * FROM get_order_history('Nicolas Pelaez')

---

## 3. Retrieve Company Policies
- **Action**: Access the internal knowledge base or policy documents related to returns, refunds, and exchanges.  
- **Why**: Verifying you’re in compliance with company guidelines prevents potential errors and conflicts.

---

In [0]:
%sql
SELECT * FROM databricks_workshop.jywu.policies 
LIMIT 5

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_return_policy()
RETURNS TABLE (
  policy           STRING,
  policy_details   STRING,
  last_updated     DATE
)
COMMENT 'Returns the details of the Return Policy'
LANGUAGE SQL
RETURN (
  SELECT
    policy,
    policy_details,
    last_updated
  FROM databricks_workshop.jywu.policies
  WHERE policy = 'Return Policy'
  LIMIT 1
);

In [0]:
%sql
SELECT * FROM get_return_policy()

---

## 4. Python functions can be used as well! Here's an example
- **Action**: Provide a **Python function** that can supply the Large Language Model (LLM) with the current date.  
- **Why**: Automating date retrieval helps in scheduling pickups, refund timelines, and communication deadlines.

###### Note: For this lab we will not be using this function but leaving as example.
---

In [0]:
def get_todays_date() -> str:
    """
    Returns today's date in 'YYYY-MM-DD' format.

    Returns:
        str: Today's date in 'YYYY-MM-DD' format.
    """
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

In [0]:
today = get_todays_date()
today

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

# this will deploy the tool to UC, automatically setting the metadata in UC based on the tool's docstring & typing hints
python_tool_uc_info = client.create_python_function(func=get_todays_date, catalog=catalog_name, schema=schema_name, replace=True)

# the tool will deploy to a function in UC called `{catalog}.{schema}.{func}` where {func} is the name of the function
# Print the deployed Unity Catalog function name
#print(f"Deployed Unity Catalog function name: {python_tool_uc_info.full_name}")

In [0]:
from IPython.display import display, HTML

# Retrieve the Databricks host URL
workspace_url = spark.conf.get('spark.databricks.workspaceUrl')

# Create HTML link to created functions
html_link = f'<a href="https://{workspace_url}/explore/data/functions/{catalog_name}/{schema_name}/get_order_history" target="_blank">Go to Unity Catalog to see Registered Functions</a>'
display(HTML(html_link))

## Now lets go over to the AI Playground to see how we can use these functions and assemble our first Agent!

### The AI Playground can be found on the left navigation bar under 'AI/ML' or you can use the link created below

In [0]:
# Create HTML link to AI Playground
html_link = f'<a href="https://{workspace_url}/ml/playground" target="_blank">Go to AI Playground</a>'
display(HTML(html_link))